In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime
import subprocess
import seaborn as sns

## Read in Data

The files have already been preprocessed in R in `utils.R`

Those were the parts for 1.1 and 1.2

In [ ]:
biogrid_lcc = pd.read_csv("Files/Biogrid.txt", sep='\t')
huri_lcc = pd.read_csv("Files/Huri.txt", sep='\t')
string_lcc = pd.read_csv("Files/String.txt", sep='\t')
reactome_lcc = pd.read_csv("Files/Reactome.txt", sep='\t')

interactomes_lcc = [biogrid_lcc, huri_lcc, string_lcc, reactome_lcc]

disease = pd.read_csv("Files/Cardiomyopathy.txt").iloc[:,0].tolist()

In [ ]:
biogrid_lcc.head()

Build graph of interactome LCC size to find all the nodes and links (not just disease)

In [ ]:
def create_graph(df, name):
    """
    Function th create graph

    Params :
    - df : the interactome
    - name : name of the graph

    Returns : instance of nxGraph
    """
    # Create instance of graph
    graph = nx.Graph(name = name)

    edges = []
    for i in range(0,df.shape[0]):
        # For each row, we add it as an edge
        edges.append((df.iloc[i,0],df.iloc[i,1]))
    
    # Get list of *unique* genes among "GeneA" and "GeneB" columns
    nodes = list(set(df["GeneA"]) | set(df["GeneB"]))

    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)

    return graph

In [ ]:
print("Running for ~3 minutes...")

biogrid_lcc_graph = create_graph(biogrid_lcc, "biogrid_interactions")
huri_lcc_graph = create_graph(huri_lcc, "huri_interactions")
string_lcc_graph = create_graph(string_lcc, "string_interactions")
reactome_lcc_graph = create_graph(reactome_lcc, "reactome_interactions")

### 1.3. Compute and characterize the disease LCC 

1.3.1 Check for the presence of disease genes in the LCCs of each interactome and identify the disease interactomes by getting the
interactions among disease genes

In [ ]:
# 1. We get the set of disease genes included in "GeneA" and "GeneB" for each interactome
# 2. Then we create a new "sub-interactome" made only of disease genes interactions

biogrid_filter = biogrid_lcc["GeneA"].isin(disease) & biogrid_lcc["GeneB"].isin(disease)
biogrid_disease_interactions = biogrid_lcc[biogrid_filter].reset_index(drop=True)

huri_filter = huri_lcc["GeneA"].isin(disease) & huri_lcc["GeneB"].isin(disease)
huri_disease_interactions = huri_lcc[huri_filter].reset_index(drop=True)

string_filter = string_lcc["GeneA"].isin(disease) & string_lcc["GeneB"].isin(disease)
string_disease_interactions = string_lcc[string_filter].reset_index(drop=True)

reactome_filter = reactome_lcc["GeneA"].isin(disease) & reactome_lcc["GeneB"].isin(disease)
reactome_disease_interactions = reactome_lcc[reactome_filter].reset_index(drop=True)

In [ ]:
# We create a specific graph for the diseases interactions
biogrid_disease_graph = create_graph(biogrid_disease_interactions, "biogrid_disease_interactions")
huri_disease_graph = create_graph(huri_disease_interactions, "huri_disease_interactions")
string_disease_graph = create_graph(string_disease_interactions, "string_disease_interactions")
reactome_disease_graph = create_graph(reactome_disease_interactions, "reactome_disease_interactions")

In [ ]:
## Visualizing Biogrid disease graph
nx.draw(biogrid_disease_graph, with_labels=True)
plt.show()

1.3.2 Summarize the Interactome and GDA-related data as in table 1

In [ ]:
table1_dict_loop = {
    'biogrid': {},
    'huri': {},
    'string': {},
    'reactome': {}
}

graphs = {
    'biogrid': (biogrid_lcc_graph, biogrid_disease_graph),
    'huri': (huri_lcc_graph, huri_disease_graph),
    'string': (string_lcc_graph, string_disease_graph),
    'reactome': (reactome_lcc_graph, reactome_disease_graph)
}

for db_name, (lcc_graph, disease_graph) in graphs.items():
    # 1. Interactome size
    table1_dict_loop[db_name]["nodes_edges"] = (lcc_graph.number_of_nodes(), lcc_graph.number_of_edges())
    
    # 2. Number of disease genes
    table1_dict_loop[db_name]["nbr_disease_genes"] = disease_graph.number_of_nodes()
    
    # 3. Percentage of disease genes
    table1_dict_loop[db_name]["percent"] = round((disease_graph.number_of_nodes() / lcc_graph.number_of_nodes()) * 100, 2)
    
    # 4. Disease LCC size
    largest_cc = max(nx.connected_components(disease_graph), key=len)
    table1_dict_loop[db_name]["disease_lcc_size"] = len(largest_cc)

table1_dict_loop

1.3.3 Compute those metrics on the largest disease LCC :
- Node degree
- Betweenness centrality
- Eigenvector centrality
- Closeness centrality
- ratio Betweenness/Node degree

In [ ]:
# Find the largest disease LCC across all databases
largest_LCC_size = max(table1_dict_loop[db]["disease_lcc_size"] for db in table1_dict_loop)
largest_LCC_name = max(table1_dict_loop, key=lambda db: table1_dict_loop[db]["disease_lcc_size"])

# Sanity check - verify all LCC sizes are <= the largest
for db_name in table1_dict_loop:
    assert table1_dict_loop[db_name]["disease_lcc_size"] <= largest_LCC_size

print(f"Largest disease LCC: {largest_LCC_name} with {largest_LCC_size} nodes")

disease_graph = graphs[largest_LCC_name][1]
largest_cc = max(nx.connected_components(disease_graph), key=len)
largest_LCC = disease_graph.subgraph(largest_cc)

We note that the largest disease LCC is from the String interactome

In [ ]:
def call_time_infos(starttime, metric):
    endtime = datetime.now()
    elapsedtime = endtime - starttime
    print(f"Elapsed time to {metric} computation: " + str(elapsedtime))

In [ ]:
starttime = datetime.now()

# 1. Node degree
degrees = largest_LCC.degree()
call_time_infos(starttime, "degree")

# 2. Betweenness centrality
betweenness = nx.betweenness_centrality(largest_LCC)
call_time_infos(starttime, "betweenness")

# 3. Closeness centrality
closeness = nx.closeness_centrality(largest_LCC)
call_time_infos(starttime, "closeness")

# 4. Eigenvector centrality
eigen = nx.eigenvector_centrality(largest_LCC)
call_time_infos(starttime, "eigenvector centrality")

# Stop timing
endtime = datetime.now()
elapsedtime = endtime - starttime
print("Total elapsed time: " + str(elapsedtime) + '\n')

# build a dataframe with all metrics
metrics_largest_LCC = pd.DataFrame({
    "Node": list(largest_LCC.nodes()),
    "degree": [degrees[n] for n in largest_LCC],
    "betweenness": [betweenness[n] for n in largest_LCC],
    "closeness": [closeness[n] for n in largest_LCC],
    "eigenvector_centrality": [eigen[n] for n in largest_LCC],
    "ratio_betweenness_degree": [betweenness[n]/degrees[n] for n in largest_LCC]
})

# Sort by degree descending
metrics_largest_LCC = metrics_largest_LCC.sort_values(by="degree", ascending=False).reset_index(drop=True)

metrics_largest_LCC

1.3.4 Report in a table the above network measures of the first 20 disease genes

In [ ]:
# Print header once
top20_disease_genes_degrees = metrics_largest_LCC.iloc[0:20,:]
top20_disease_genes_degrees

1.3.5 Represent node degree and node betweenness in a scatterplot

In [ ]:
degree_values = [d for d in metrics_largest_LCC['degree'].tolist()]
betweenness_values = [d for d in metrics_largest_LCC['betweenness'].tolist()]

# TODO : See if this scatter is relevant
plt.scatter(range(len(degree_values)), degree_values, alpha=0.6)
plt.xlabel("Nodes indexes (ranked as those on the right have smaller degree)")
plt.ylabel("Degree")
plt.title("Node degree scatter plot (disease LCC)")
plt.show()

# TODO : See if this scatter is relevant
plt.scatter(range(len(betweenness_values)), betweenness_values, alpha=0.6)
plt.xlabel("Nodes indexes (ranked as those on the right have smaller degree)")
plt.ylabel("Betweenness centrality")
plt.title("Node betweenness centrality scatter plot (disease LCC)")
plt.show()

# TODO : check (ask the prof ?) if this scatter is the one he wants ?
plt.scatter(degree_values, betweenness_values, alpha=0.6)
for node in metrics_largest_LCC['Node']:
    # We add gene symbols on the graph for significative genes (in terms of betweenness)
    if betweenness[node] > 0.01:
        plt.text(degrees[node], betweenness[node], node, fontsize=8)
plt.xlabel("Node degree")
plt.ylabel("Betweenness centrality")
plt.title("Degree VS Betweenness centrality scatter plot (disease LCC)")
plt.show()

As we can see, there is a correlation between node degree and betweenness centrality. The more connected is the node, the geater is the betweenness centrality.

Betweenness centrality is the measure of how much a node is a "bridge/bottleneck" in other node pathways.

The most interesting nodes would be those with low degree and high betweenness ! Unfortunately here this scenario is doesn't appear. Maybe for `VCP` gene but not enough evidence...

## 2. Comparative analysis of disease genes identification algorithms

### 2.1 Use algorithms to infer and then validate putative disease genes.

2.1.1 DIAMOnD, default parameters

In [ ]:
# With n=200, running for ~1 minute
!python DIAMOnD.py Files/Biogrid.txt Files/Cardiomyopathy.txt 200 Files/diamond_output_200.txt

2.1.2 Diffusion-based, diffusion times: t=[0.1, 1.0, 2.0, 5.0, 10.0]

In [ ]:
# With t=0.1, running for ~4 seconds
!python HeatDiffusion.py Files/Biogrid.txt Files/Cardiomyopathy.txt 200 0.1 Files/heat_diffusion_output_0.1.txt

2.1.3 OPTIONAL : Run Proconsul

### 2.2 Computational validation

2.2.1 For each interactome: perform a 5-fold cross validation in the following way

Split the disease genes set $S_0$ into 5 subsets.

For each fold, select one subset as probe set $S_P$ and the remaining four subsets as training set $S_T$.

For every fold, run both algorithms on all four interactomes (8 combinations, 40 runs total) using the training $S_T$ sets.

In [ ]:
def run_algo(algo, network, train_set, output_file, fold_idx, n, t=None):
    """
    Run DIAMOnD algorithm on the selected fold, save prediction.

    Params :
    - algo : algorithm to run ["diamond" | "heatdiffusion"]
    - network : interactome
    - train_set : subset of disease gene
    - output_file : name/path for the saved file
    - fold_idx : index of the current fold
    - n : number of disease genes to predict
    - t : diffusion times (for heatdiffusion)
    """

    if algo=="diamond":
        print("Running Diamond for ~1 minute")
        diamond_script = "DIAMOnD.py"
        cmd = ['python', diamond_script, network, train_set, str(n), output_file]
    elif algo=="heatdiffusion":
        print("Running heat diffusion for few seconds")
        diamond_script = "HeatDiffusion.py"
        assert(t!=None)
        cmd = ['python', diamond_script, network, train_set, str(n), str(t), output_file]
    else:
        raise AttributeError(algo+" not found. Shoud be 'diamond' or 'heatdiffusion'.")

    print(f"Running: {' '.join(cmd)}")
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(f"✓ Fold {fold_idx} completed successfully")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"✗ Error in Fold {fold_idx}")
        print(e.stderr)

In [ ]:
from sklearn.model_selection import KFold

def run_algo_on_one_interactome(interactome, disease, K, n, try_multiple_t=False):
    """
    Create K-folds and run algorithms for the specified interactome for the K folds.
    
    Parameters :
    - interactome : path to the interactome file
    - disease : set of disease genes
    - K : number of folds
    - n : number of genes to predict
    - try_multiple_t : if testing different values of diffusion time
    """

    # Load folds
    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    # Iterate on each fold (train, test)
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(disease), 1):
        print(f"Processing Fold {fold_idx}")
        
        # Get the disease genes from the current train and test fold
        train_genes = [disease[i] for i in train_idx]
        test_genes = [disease[i] for i in test_idx]
        train_file = f'Files/Folds/train_fold_{fold_idx}.txt'
        test_file = f'Files/Folds/test_fold_{fold_idx}.txt'
        
        # Save train (for algorithms)
        with open(train_file, 'w') as f:
            f.write('\n'.join(train_genes))
        
        # Save test (to evaluate)
        with open(test_file, 'w') as f:
            f.write('\n'.join(test_genes))

        # Diamond
        run_algo("diamond", interactome, train_file, f'Files/Folds/{interactome.removesuffix(".txt").removeprefix("Files/")}_diamond_results_fold_{fold_idx}.txt', fold_idx, n)

        # Heat diffusion
        if try_multiple_t:
            t = [0.1, 1.0, 2.0, 5.0, 10.0]
            for time in t:
                output_file = f'Files/Folds/{interactome.removesuffix(".txt").removeprefix("Files/")}_heatdiffusion_results_fold_{fold_idx}_{time}.txt'
                run_algo("heatdiffusion", interactome, train_file, output_file, fold_idx, n, t=time)
        else:
            t = 1.0 # time diffusion selected after comparing in 'try_multiple_t' mode
            output_file = f'Files/Folds/{interactome.removesuffix(".txt").removeprefix("Files/")}_heatdiffusion_results_fold_{fold_idx}_{t}.txt'
            run_algo("heatdiffusion", interactome, train_file, output_file, fold_idx, n, t=t)

In [ ]:
print("Running for ~32 minutes...")

run_algo_on_one_interactome("Files/Biogrid.txt", disease, K=5, n=200, try_multiple_t=False)
run_algo_on_one_interactome("Files/Huri.txt", disease, K=5, n=200, try_multiple_t=False)
run_algo_on_one_interactome("Files/String.txt", disease, K=5, n=200, try_multiple_t=False)
run_algo_on_one_interactome("Files/Reactome.txt", disease, K=5, n=200, try_multiple_t=False)

Evaluate each run by checking how many genes from the probe set $S_P$ appear in the algorithm’s output.

In [ ]:
def evaluate_predictions(prediction_file, true_file, top_X=None):
    """
    Evaluate the algorithm by comparing the expected genes from test_set with the set of genes in the result_file
    
    Parameters :
    - prediction_file : path to the file containing the predictions of disease genes obtained after running the algorithms
    - true_genes_file : set of genes expected in the result_file
    - top_X : to evalute only on top X genes in prediction_file

    Returns :
    - Dictionnary with all the metrics
    """

    # Creates a set with the real disease genes
    true_genes = {line.strip() for line in open(true_file)}

    # If top_X not provided, we analyse the whole file (top_X = nb lines)
    if top_X==None:
        with open(prediction_file) as f:
            top_X = len(f.readlines())

    # Creates a set with the predicted genes
    predicted_genes = {
        line.strip().split('\t')[1]
        for i, line in enumerate(open(prediction_file))
        if not line.startswith('#') and i < top_X # We skip the first line (with "#") and stop until top_X
    }

    # Compute true positive, false positive and false negative for the metrics
    TP = len(predicted_genes & true_genes)
    FP = len(predicted_genes - true_genes)
    FN = len(true_genes - predicted_genes)

    # Compute required metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'TP': TP,
        'FP': FP,
        'FN': FN
    }


In [ ]:
def iterate_prediction(top_X=None):
    interactomes = ["Biogrid", "Huri", "Reactome", "String"]
    results_data = []

    # Iterate on each fold
    for fold in range(1, 6):
        # Iterate on each interactome
        for interactome in interactomes:
            for X in (top_X if top_X is not None else [None]):
                # Diamond
                results_diamond = evaluate_predictions(
                    prediction_file=f'Files/Folds/{interactome}_diamond_results_fold_{fold}.txt',
                    true_file=f'Files/Folds/test_fold_{fold}.txt',
                    top_X=X
                )
                results_data.append({
                    'Top_X': X,
                    'Fold': fold,
                    'Interactome': interactome,
                    'Algorithm': 'Diamond',
                    'Precision': results_diamond['precision'],
                    'Recall': results_diamond['recall'],
                    'F1': results_diamond['f1'],
                    'TP': results_diamond['TP'],
                    'FN': results_diamond['FN']
                })
                
                # Heat Diffusion
                results_heat = evaluate_predictions(
                    prediction_file=f'Files/Folds/{interactome}_heatdiffusion_results_fold_{fold}_1.0.txt',
                    true_file=f'Files/Folds/test_fold_{fold}.txt',
                    top_X=X
                )
                results_data.append({
                        'Top_X': X,
                        'Fold': fold,
                        'Interactome': interactome,
                        'Algorithm': 'HeatDiffusion',
                        'Precision': results_heat['precision'],
                        'Recall': results_heat['recall'],
                        'F1': results_heat['f1'],
                        'TP': results_heat['TP'],
                        'FN': results_heat['FN']
                    })
                
    return results_data

In [ ]:
def print_summary_metrics(df):
    # Comparing algorithms and interactomes
    algo_comparison = df.groupby('Algorithm')['Recall'].agg(['mean', 'std', 'min', 'max'])
    algo_comparison['mean_pct'] = algo_comparison['mean'] * 100
    algo_comparison['std_pct'] = algo_comparison['std'] * 100

    interactome_comparison = df.groupby('Interactome')['Recall'].agg(['mean', 'std', 'min', 'max'])
    interactome_comparison['mean_pct'] = interactome_comparison['mean'] * 100
    interactome_comparison['std_pct'] = interactome_comparison['std'] * 100
    interactome_comparison = interactome_comparison.sort_values('mean_pct', ascending=False)

    # ============================================================================
    # Executive summary
    # ============================================================================
    print("\n" + "="*80)
    print("EXECUTIVE SUMMARY")
    print("="*80)

    best_algo = algo_comparison['mean_pct'].idxmax()
    best_algo_recall = algo_comparison.loc[best_algo, 'mean_pct']

    best_interactome_overall = interactome_comparison['mean_pct'].idxmax()
    best_interactome_recall = interactome_comparison.loc[best_interactome_overall, 'mean_pct']

    # Best combinaison
    best_combo = df.groupby(['Algorithm', 'Interactome'])['Recall'].mean().idxmax()
    best_combo_recall = df.groupby(['Algorithm', 'Interactome'])['Recall'].mean().max() * 100

    print(f"""
    1. Best Algorithm:     {best_algo} ({best_algo_recall:.2f}% average recall)
    2. Best Interactome:   {best_interactome_overall} ({best_interactome_recall:.2f}% average recall)
    3. Best Combination:   {best_combo[0]} + {best_combo[1]} ({best_combo_recall:.2f}% recall)
    """)

    # ============================================================================
    # Visualizing
    # ============================================================================
    fig, axes = plt.subplots(3, 1, figsize=(10, 10))

    # Plot 1: Barplot - Algorithmes
    algo_means = df.groupby('Algorithm')['Recall'].mean() * 100
    axes[0].bar(algo_means.index, algo_means.values, color=['blue', 'red'])
    axes[0].set_ylabel('Average Recall (%)', fontsize=12)
    axes[0].set_title('Algorithm Comparison', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    for i, v in enumerate(algo_means.values):
        axes[0].text(i, v + 1, f'{v:.2f}%', ha='center', fontweight='bold')

    # Plot 2: Barplot - Interactomes
    interactome_means = df.groupby('Interactome')['Recall'].mean().sort_values(ascending=False) * 100
    axes[1].bar(interactome_means.index, interactome_means.values, 
                color=['green', 'orange', 'purple', 'blue'])
    axes[1].set_ylabel('Average Recall (%)', fontsize=12)
    axes[1].set_title('Interactome Comparison', fontsize=14, fontweight='bold')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(axis='y', alpha=0.3)
    for i, v in enumerate(interactome_means.values):
        axes[1].text(i, v + 1, f'{v:.2f}%', ha='center', fontweight='bold')

    # Plot 3: Heatmap - Algorithm × Interactome
    heatmap_data = df.pivot_table(values='Recall', index='Interactome', 
                                columns='Algorithm', aggfunc='mean') * 100
    sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='YlOrRd', 
                ax=axes[2], cbar_kws={'label': 'Recall (%)'})
    axes[2].set_title('Recall Heatmap: Algorithm × Interactome', 
                        fontsize=14, fontweight='bold')

    plt.tight_layout()

In [ ]:
# Create dataframe

results_data = iterate_prediction(top_X=None)
df = pd.DataFrame(results_data)

# Create a dataframe specific to display only what's ask in this questions
display_df = df.assign(**{
        'Disease genes found': df['TP'],
        'Total disease genes': df['TP'] + df['FN']
    })[['Fold','Interactome','Algorithm','Disease genes found','Total disease genes']]

print("\n" + "="*80)
print("DETAILED RESULTS (all folds)")
print("="*80)
print(display_df.to_string(index=False))

2.2.2 Compute the following performance metrics for all the combinations algorithm/interactome:
- precision (average SD)
- recall (average SD)
- F1-score (average SD)

Precision : $$\frac{\text{True positive}}{\text{True Positive + False Positive}}$$

Recall : $$\frac{\text{True positive}}{\text{True Positive + False Negative}}$$

F1-Score : $$ 2 * \frac{\text{precision} * \text{recall}}{\text{precision + recall}}$$

- **True positive** corresponds to predicted real disease genes
- **False positive** corresponds to predicted not disease genes
- **False negative** corresponds to the disease genes not found

In [ ]:
metrics_summary = (
    df
    .groupby(['Algorithm', 'Interactome'])
    .agg({
        'Precision': ['mean', 'std'],
        'Recall': ['mean', 'std'],
        'F1': ['mean', 'std']
    })
)

# Display
metrics_summary.columns = [
    f"{metric}_{stat}"
    for metric, stat in metrics_summary.columns
]

metrics_summary = metrics_summary.reset_index()

metrics_summary

In [ ]:
print_summary_metrics(df)

In conclusion, the best combinasion is using String interactome with the heat diffusion algorithm.

2.2.3 Provide the performance measures selecting the top 50 positions and the top X positions where $X$ = $\frac{n}{8}$, $\frac{n}{4}$, $\frac{n}{2}$, $n$, with n=number of known GDAs (i.e., number of disease’s seed genes)

In our case we have n=373 disease genes in $S_0$. So we'll keep the following top_X :
- $X = 50$
- $X = \frac{n}{8} = 47$
- $X = \frac{n}{4} = 93$
- $X = \frac{n}{2} = 186$
- $X = n = 373$

In [ ]:
n = len(disease) # n = number of total disease gene set (S_0)
top_X = [50, round(n/8), round(n/4), round(n/2), n]

results_data_topX = iterate_prediction(top_X=top_X)
# Create dataframe
df_topX = pd.DataFrame(results_data_topX)

metrics_summary_topX = (
    df_topX
    .groupby(['Top_X', 'Algorithm', 'Interactome'])
    .agg({
        'Precision': ['mean', 'std'],
        'Recall': ['mean', 'std'],
        'F1': ['mean', 'std']
    })
)

# Display
metrics_summary_topX.columns = [
    f"{metric}_{stat}"
    for metric, stat in metrics_summary_topX.columns
]

metrics_summary_topX = metrics_summary_topX.reset_index()

metrics_summary_topX

In [ ]:
print_summary_metrics(df_topX)

In conclusion, even after evaluating on several top_X predicted genes, we still have the best combination : String interactome with heat diffusion algorithm.

## 3. Putative disease gene identification

### 3.1.  According to the performance metrics obtained in the validation phase at point 2: 

3.1.1 select the best performing algorithm and the best interactome and apply the 
process  to  predict  new  putative  disease  genes  using  all known GDAs as 
seed genes 

In [ ]:
import os

def run_heatdiffusion_all_seeds(interactome_file, disease_genes, output_file, t=1.0):
    """
    Run HeatDiffusion algorithm using ALL disease genes as seeds.
    Uses the existing HeatDiffusion.py script.
    
    Parameters:
    - interactome_file: path to interactome file (e.g., 'Files/String.txt')
    - disease_genes: list of all disease gene names
    - output_file: where to save the results
    - t: heat diffusion time parameter (default=1.0)
    """
    import subprocess
    
    print("="*80)
    print("Running HeatDiffusion with ALL Seeds")
    print("="*80)
    print(f"Interactome: {interactome_file}")
    print(f"Number of seed genes: {len(disease_genes)}")
    print(f"Time parameter t: {t}")
    print(f"Output file: {output_file}")
    
    # Create temporary seeds file with ALL disease genes
    seeds_file = "Files/all_seeds_temp.txt"
    with open(seeds_file, 'w') as f:
        f.write('\n'.join(disease_genes))
    
    print(f"\n✓ Created seeds file with {len(disease_genes)} genes")
    
    # Run HeatDiffusion using existing script
    # Note: n should be large enough to get all genes in the network
    # We'll use a large number (e.g., 20000) to ensure we get all predictions
    n = 20000  # Large number to get all predictions
    
    heatdiffusion_script = "HeatDiffusion.py"
    cmd = ['python', heatdiffusion_script, interactome_file, seeds_file, str(n), str(t), output_file]
    
    print(f"\nRunning: {' '.join(cmd)}")
    print("(This may take a few minutes...)\n")
    
    try:
        result = subprocess.run(cmd, check=True, capture_output=True, text=True, timeout=600)
        print("✓ HeatDiffusion completed successfully!")
        print(result.stdout)
        
        # Verify output file exists
        if os.path.exists(output_file):
            # Load and display results
            df = pd.read_csv(output_file, sep='\t', header=None, names=['Gene', 'Score'])
            print(f"✓ Generated predictions for {len(df)} genes")
            print(f"\nTop 20 predicted genes:")
            print(df.head(20))
            return df
        else:
            print(f"✗ Output file not created: {output_file}")
            return None
            
    except subprocess.CalledProcessError as e:
        print(f"✗ HeatDiffusion failed with error:")
        print(e.stderr)
        return None
    except FileNotFoundError:
        print(f"\n✗ HeatDiffusion.py script not found!")
        print("Make sure HeatDiffusion.py is in the current directory")
        return None
    except Exception as e:
        print(f"\n✗ Error running HeatDiffusion: {e}")
        return None


In [ ]:
print(f"Running HeatDiffusion with {len(disease)} seed genes...")

start_time = datetime.now()

output_file = "Files/String_heatdiffusion_all_seeds.txt"

heatdiffusion_results = run_heatdiffusion_all_seeds(
    interactome_file="Files/String.txt",
    disease_genes=disease,
    output_file=output_file,
    t=1.0
)

end_time = datetime.now()
print(f"\nTime elapsed: {end_time - start_time}")

In [ ]:
def get_putative_genes(predictions_df, known_disease_genes, top_n=100):
    """
    Extract top N putative disease genes from predictions.
    Excludes known disease genes.
    
    Parameters:
    - predictions_df: DataFrame with 'Gene' and 'Score' columns
    - known_disease_genes: list of known disease genes to exclude
    - top_n: number of putative genes to return
    
    Returns:
    - list of putative gene names
    - DataFrame with putative genes and scores
    """
    print("="*80)
    print(f"Extracting Top {top_n} Putative Disease Genes")
    print("="*80)
    
    # Filter out known disease genes
    putative_df = predictions_df[~predictions_df['Gene'].isin(known_disease_genes)].copy()
    
    print(f"Total predictions: {len(predictions_df)}")
    print(f"After removing {len(known_disease_genes)} known disease genes: {len(putative_df)}")
    
    # Get top N
    top_putative = putative_df.head(top_n).reset_index(drop=True)
    
    print(f"\nTop 10 putative disease genes:")
    print(top_putative.head(10))
    
    return top_putative['Gene'].tolist(), top_putative

In [ ]:
putative_genes, putative_df = get_putative_genes(
    heatdiffusion_results,
    disease,
    top_n=100
)

putative_df.to_csv("Files/putative_genes_top100.csv", index=False)
print(f"\n✓ Saved top 100 putative genes to Files/putative_genes_top100.csv")

3.2.1: Enrichment Analysis for Putative Disease Genes

In [ ]:
import requests
import json
import time

def enrichr_analysis(gene_list, gene_set_libraries, description=""):
    """
    Perform enrichment analysis using EnrichR API.
    
    Parameters:
    - gene_list: list of gene symbols
    - gene_set_libraries: list of library names
    - description: description for the gene list
    
    Returns:
    - dictionary with results for each library
    """
    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr'
    
    # Submit gene list
    genes_str = '\n'.join(gene_list)
    payload = {
        'list': (None, genes_str),
        'description': (None, description)
    }
    
    print(f"\n{'='*80}")
    print(f"EnrichR Analysis: {description}")
    print(f"{'='*80}")
    print(f"Submitting {len(gene_list)} genes...")
    
    response = requests.post(f'{ENRICHR_URL}/addList', files=payload)
    
    if not response.ok:
        raise Exception(f'Error analyzing gene list: {response.text}')
    
    data = json.loads(response.text)
    user_list_id = data['userListId']
    print(f"✓ List ID: {user_list_id}")
    
    # Get enrichment results for each library
    all_results = {}
    
    for library in gene_set_libraries:
        print(f"\nQuerying {library}...")
        time.sleep(0.5)  # Rate limiting
        
        response = requests.get(
            f'{ENRICHR_URL}/enrich',
            params={
                'userListId': user_list_id,
                'backgroundType': library
            }
        )
        
        if not response.ok:
            print(f"⚠ Warning: Could not get results for {library}")
            continue
        
        data = json.loads(response.text)
        
        if library not in data or len(data[library]) == 0:
            print(f"  No results for {library}")
            all_results[library] = pd.DataFrame()
            continue
        
        # Parse results
        results = data[library]
        
        # Convert to DataFrame
        df = pd.DataFrame(results, columns=[
            'Rank', 'Term', 'P-value', 'Z-score', 'Combined Score',
            'Overlapping Genes', 'Adjusted P-value', 'Old P-value', 'Old Adjusted P-value'
        ])
        
        # Filter for significant results (adjusted p-value < 0.05)
        df_sig = df[df['Adjusted P-value'] < 0.05].copy()
        
        print(f"  ✓ Found {len(df_sig)} significant terms (adj p-value < 0.05)")
        
        all_results[library] = df_sig
    
    return all_results

In [ ]:
libraries = [
    'GO_Biological_Process_2023',
    'GO_Molecular_Function_2023',
    'GO_Cellular_Component_2023',
    'Reactome_2022',
    'KEGG_2021_Human'
]

print("Gene Set Libraries:")
for i, lib in enumerate(libraries, 1):
    print(f"  {i}. {lib}")

In [ ]:
putative_enrichment = enrichr_analysis(
    putative_genes,
    libraries,
    description="Cardiomyopathy_Putative_Genes"
)

3.2.2: Enrichment Analysis for Original Disease Genes

In [ ]:

original_enrichment = enrichr_analysis(
    disease,
    libraries,
    description="Cardiomyopathy_Original_Genes"
)

3.2.3: Compare Enriched Functions (Overlap Analysis)

In [ ]:
def compare_enrichment_results(original_results, putative_results, library_name):
    """
    Compare enrichment results between original and putative disease genes.
    
    Returns:
    - Dictionary with overlap analysis results
    """
    print(f"\n{'='*80}")
    print(f"Comparing Enrichment: {library_name}")
    print(f"{'='*80}")
    
    if library_name not in original_results or library_name not in putative_results:
        print(f"⚠ Results not available for {library_name}")
        return None
    
    orig_df = original_results[library_name]
    put_df = putative_results[library_name]
    
    if orig_df.empty or put_df.empty:
        print(f"⚠ No significant results in one or both gene sets")
        return None
    
    # Get term names
    orig_terms = set(orig_df['Term'].tolist())
    put_terms = set(put_df['Term'].tolist())
    
    # Find overlap
    overlap_terms = orig_terms & put_terms
    orig_only = orig_terms - put_terms
    put_only = put_terms - orig_terms
    
    print(f"\nOriginal disease genes: {len(orig_terms)} significant terms")
    print(f"Putative disease genes: {len(put_terms)} significant terms")
    print(f"Overlapping terms: {len(overlap_terms)}")
    print(f"Original only: {len(orig_only)}")
    print(f"Putative only: {len(put_only)}")
    
    if len(overlap_terms) > 0:
        # Calculate overlap percentage
        overlap_pct = (len(overlap_terms) / min(len(orig_terms), len(put_terms))) * 100
        print(f"Overlap percentage: {overlap_pct:.1f}%")
        
        print(f"\n{'-'*80}")
        print("Top 10 Overlapping Enriched Terms:")
        print(f"{'-'*80}")
        
        # Create comparison DataFrame
        overlap_list = []
        for term in overlap_terms:
            orig_row = orig_df[orig_df['Term'] == term].iloc[0]
            put_row = put_df[put_df['Term'] == term].iloc[0]
            
            overlap_list.append({
                'Term': term,
                'Original_P_adj': orig_row['Adjusted P-value'],
                'Putative_P_adj': put_row['Adjusted P-value'],
                'Original_Combined_Score': orig_row['Combined Score'],
                'Putative_Combined_Score': put_row['Combined Score']
            })
        
        overlap_df = pd.DataFrame(overlap_list)
        overlap_df = overlap_df.sort_values('Original_P_adj')
        
        # Display top 10
        print(overlap_df.head(10).to_string(index=False))
        
        return {
            'overlap_df': overlap_df,
            'orig_terms': orig_terms,
            'put_terms': put_terms,
            'overlap_terms': overlap_terms,
            'overlap_count': len(overlap_terms),
            'overlap_pct': overlap_pct
        }
    else:
        print("No overlapping terms found")
        return None

In [ ]:
!pip install matplotlib-venn

In [ ]:
import matplotlib_venn

comparison_results = {}

for library in libraries:
    comparison = compare_enrichment_results(
        original_enrichment,
        putative_enrichment,
        library
    )
    
    if comparison is not None:
        comparison_results[library] = comparison

def plot_venn_diagram(comparison, library_name):
    """Create Venn diagram showing overlap of enriched terms."""
    if comparison is None:
        return None
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    venn = matplotlib_venn.venn2(
        subsets=(
            len(comparison['orig_terms']) - len(comparison['overlap_terms']),
            len(comparison['put_terms']) - len(comparison['overlap_terms']),
            len(comparison['overlap_terms'])
        ),
        set_labels=('Original Disease Genes', 'Putative Disease Genes'),
        ax=ax
    )
    
    # Customize colors
    if venn.get_patch_by_id('10'):
        venn.get_patch_by_id('10').set_color('lightblue')
        venn.get_patch_by_id('10').set_alpha(0.6)
    if venn.get_patch_by_id('01'):
        venn.get_patch_by_id('01').set_color('lightcoral')
        venn.get_patch_by_id('01').set_alpha(0.6)
    if venn.get_patch_by_id('11'):
        venn.get_patch_by_id('11').set_color('lightgreen')
        venn.get_patch_by_id('11').set_alpha(0.7)
    
    plt.title(f'Enriched Terms Overlap\n{library_name}', 
              fontsize=14, fontweight='bold', pad=20)
    
    return fig

print("\nGenerating Venn diagrams...")

for library in comparison_results.keys():
    fig = plot_venn_diagram(comparison_results[library], library)
    if fig:
        plt.tight_layout()
        plt.show()

In [ ]:

print("\n" + "="*80)
print("FINAL SUMMARY: ENRICHMENT OVERLAP ANALYSIS")
print("="*80)

summary_data = []

for library in libraries:
    if library in original_enrichment and not original_enrichment[library].empty:
        orig_count = len(original_enrichment[library])
    else:
        orig_count = 0
    
    if library in putative_enrichment and not putative_enrichment[library].empty:
        put_count = len(putative_enrichment[library])
    else:
        put_count = 0
    
    if library in comparison_results:
        overlap_count = comparison_results[library]['overlap_count']
        overlap_pct = comparison_results[library]['overlap_pct']
    else:
        overlap_count = 0
        overlap_pct = 0.0
    
    summary_data.append({
        'Library': library.replace('_', ' '),
        'Original Genes\nEnriched Terms': orig_count,
        'Putative Genes\nEnriched Terms': put_count,
        'Overlapping\nTerms': overlap_count,
        'Overlap %': f"{overlap_pct:.1f}%"
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + summary_df.to_string(index=False))

## 4. Drug repurposing

In [ ]:
# TODO (Annet ?)